In [2]:
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import os
from PIL import Image
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import v2
import torch
from torch.utils.data import DataLoader
import opendatasets as od
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from numba import cuda
from torch.utils.data import Dataset,DataLoader,Subset,random_split


In [9]:
od.download('https://www.kaggle.com/datasets/alexattia/the-simpsons-characters-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  pekpuch


Your Kaggle Key:

  ········


100%|██████████| 1.08G/1.08G [14:43<00:00, 1.31MB/s]


In [3]:
from torchvision import transforms
from torchvision import datasets

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform, target_transform=None):
      self.img_labels = os.listdir(path = img_dir)
      self.labels_map = {}
      it = 0
      for i in self.img_labels:
        self.labels_map[i] = it
        it+=1
      self.labels = []
      self.img_list = []
      for i in self.img_labels:
        img_dir_tmp = img_dir + '/' + i + '/'
        for j in os.listdir(path = img_dir_tmp):
          self.img_list.append(img_dir_tmp+j)
          self.labels.append(i)
      self.labels_final = []
      for i in self.labels:
        self.labels_final.append(self.labels_map[i])
      self.transform = transform
      self.target_transform = target_transform
    def __len__(self):
      return len(self.img_list)
    def __getitem__(self, idx):
      img_path = self.img_list[idx]
      image = Image.open(img_path).convert('RGB')
      label = self.labels_final[idx]
      w, h = image.size
      if self.transform:
        image = self.transform(image)
      return image, label

class TestDataset(Dataset):
    def __init__(self, labels, test_dir, transform):
      self.labels_map = labels
      self.test_labels_map = {}
      self.test_img_list_dir = os.listdir(path = test_dir)
      self.test_paths = []
      for i in self.test_img_list_dir:
        path = '/content/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/' + i
        self.test_paths.append(path)
        i = i.rstrip("jpg")
        indx = len(i) - 1
        while (i[indx].isalpha() != True):
          i = i[:-1]
          indx-=1
        self.test_labels_map[path] = self.labels_map[i]
      self.transform = transform
    def __len__(self):
      return len(self.test_paths)
    def __getitem__(self, idx):
      img_path = self.test_paths[idx]
      image = Image.open(img_path).convert('RGB')
      label = self.test_labels_map[img_path]
      w, h = image.size
      if self.transform:
        image = self.transform(image)
      return image, label


transforms = v2.Compose([
      v2.RandomResizedCrop(size=(224, 224), antialias=True),
      transforms.ToTensor(),
      v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

In [4]:
myDataset = CustomImageDataset (img_dir = 'the-simpsons-characters-dataset/simpsons_dataset/simpsons_dataset', transform = transforms)

In [5]:
class Net(nn.Module):
  def __init__(self, labels_count = len(myDataset.labels_map)):
    super(Net, self).__init__()

    # Сверточный слой 1
    # Этот слой применит 32 сверточных фильтра размером 3x3 на входное изображение.
    # Шаг равен 1, что означает, что фильтр будет двигаться на 1 пиксель за раз.
    # Заполнение равно 1, чтобы сохранить пространственные размеры (высоту и ширину) неизменными после операции свертки.
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)

    # Нормализация пакетов слоя 1
    # Этот слой нормализует активации нейронов в предыдущем слое, что может помочь ускорить обучение и уменьшить чувствительность к инициализации сети.
    self.batch_norm1 = nn.BatchNorm2d(32)

    # Слой максимального пулинга 1
    # Этот слой уменьшит пространственные размеры (высоту и ширину) входного объема, выбрав максимальное значение в каждом патче 2x2.
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Сверточный слой 2
    # Этот слой применит 64 сверточных фильтра размером 3x3 на выход предыдущего слоя.
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

    # Нормализация пакетов слоя 2
    self.batch_norm2 = nn.BatchNorm2d(64)

    # Слой максимального пулинга 2
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Сверточный слой 3
    # Этот слой применит 128 сверточных фильтров размером 3x3 на выход предыдущего слоя.
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

    # Нормализация пакетов слоя 3
    self.batch_norm3 = nn.BatchNorm2d(128)

    # Слой максимального пулинга 3
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Сверточный слой 4
    # Этот слой применит 256 сверточных фильтров размером 3x3 на выход предыдущего слоя.
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)

    # Нормализация пакетов слоя 4
    self.batch_norm4 = nn.BatchNorm2d(256)

    # Слой максимального пулинга 4
    self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Полносвязный слой 1
    # Этот слой соединит каждый нейрон в предыдущем слое со всеми нейронами в этом слое.
    # Он примет сплошной выход из предыдущего слоя (256 * 14 * 14) и выдаст 512 значений.
    self.fc1 = nn.Linear(256 * 14 * 14, 512)

    # Полносвязный слой 2
    # Этот слой примет выход из предыдущего слоя (512) и выдаст число значений, равное числу меток.
    self.fc2 = nn.Linear(512, labels_count)

  def forward(self, x):
    # Применить первый набор операций свертки, нормализации пакетов и максимального пулинга, за которыми следует функция активации ReLU.
    x = self.pool1(F.relu(self.batch_norm1(self.conv1(x))))
    x = self.pool2(F.relu(self.batch_norm2(self.conv2(x))))
    x = self.pool3(F.relu(self.batch_norm3(self.conv3(x))))
    x = self.pool4(F.relu(self.batch_norm4(self.conv4(x))))

    # Преобразовать выход из предыдущего слоя в единичный вектор.
    x = x.view(-1, 256 * 14 * 14)

    # Применить первый полносвязный слой, за которым следует функция активации ReLU.
    x = F.relu(self.fc1(x))

    # Применить второй полносвязный слой. Это выдаст вектор вероятностей для каждой метки.
    x = self.fc2(x)

    # Вернуть выход.
    return x


In [6]:
dataset_size = len(myDataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(myDataset, [train_size, val_size]) # Разделение Train датасета на Val и Train

In [7]:
learning_rate = 0.001
batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=myDataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size,shuffle=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = Net().to(device)

cuda


In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9,weight_decay = 0.01) # w = w - α * ∇L
criterion = nn.CrossEntropyLoss() # L = -1/N * Σ (y_i * log(p_i))

In [10]:
def model_training(model,train_loader,optimizer,criterion):
  running_loss = 0.0
  model.train()
  for inputs,labels in train_loader:
    optimizer.zero_grad() # обнуление градиентов
    inputs,labels = inputs.to(device) , labels.to(device)
    out = model(inputs) # проход батча через слои нейронной сети
    loss = criterion(out,labels) # подсчет потерь
    loss.backward() # вычисление градиентов
    optimizer.step() # обновление градиентов модели в стороны минимизации потерь
    running_loss += loss.item()

  return running_loss / len(inputs)

In [11]:
def model_validating(model,val_loader,criterion):
  running_loss = 0.0
  model.eval()
  for inputs,labels in val_loader:    
    with torch.no_grad():
      inputs,labels = inputs.to(device),labels.to(device)
      out = model(inputs)
      loss = criterion(out,labels)
      running_loss += loss.item()
  return running_loss / len(inputs)

In [12]:
print(torch.cuda.is_available())

True


In [13]:
num_epochs = 20
train_loss_arr = []
val_loss_arr = []
print(device)
for epoch in range(num_epochs):
    train_loss = model_training(model, train_loader, optimizer, criterion)
    val_loss = model_validating(model, val_loader, criterion)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    train_loss_arr.append(train_loss)
    val_loss_arr.append(val_loss)

cuda
Epoch [1/20], Train Loss: 6.1572, Val Loss: 0.7980
Epoch [2/20], Train Loss: 5.0274, Val Loss: 0.7218
Epoch [3/20], Train Loss: 4.6501, Val Loss: 0.6803
Epoch [4/20], Train Loss: 4.3419, Val Loss: 0.6310
Epoch [5/20], Train Loss: 4.0688, Val Loss: 0.6154
Epoch [6/20], Train Loss: 3.8671, Val Loss: 0.5782
Epoch [7/20], Train Loss: 3.7409, Val Loss: 0.5474
Epoch [8/20], Train Loss: 3.5536, Val Loss: 0.5286
Epoch [9/20], Train Loss: 3.4463, Val Loss: 0.5214
Epoch [10/20], Train Loss: 3.3817, Val Loss: 0.4878
Epoch [11/20], Train Loss: 3.2335, Val Loss: 0.4821
Epoch [12/20], Train Loss: 3.1691, Val Loss: 0.4603
Epoch [13/20], Train Loss: 3.1192, Val Loss: 0.4894
Epoch [14/20], Train Loss: 3.0315, Val Loss: 0.4711
Epoch [15/20], Train Loss: 2.9535, Val Loss: 0.4640
Epoch [16/20], Train Loss: 2.8945, Val Loss: 0.4260
Epoch [17/20], Train Loss: 2.8022, Val Loss: 0.4263
Epoch [18/20], Train Loss: 2.7603, Val Loss: 0.4227
Epoch [19/20], Train Loss: 2.7450, Val Loss: 0.4143
Epoch [20/20], T

In [ ]:
'''
import matplotlib.pyplot as plt
x = list(range(1, 21))
plt.plot(x, train_loss_arr)
plt.show()
plt.plot(x, val_loss_arr)
plt.show()
'''

In [1]:
testDataset = TestDataset (labels = myDataset.labels_map, test_dir = '/content/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset', transform = transforms)


NameError: name 'TestDataset' is not defined

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, '/content/model_v4.pth')

In [ ]:
checkpoint = torch.load('/content/model_v4.pth',map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
def predict_image(model,img_path):
  with open(img_path,'rb') as f :
    img = Image.open(f).convert("RGB")
  img_tensor = transforms(img).unsqueeze(0).to(device)
  out = model(img_tensor)
  pred_label = torch.argmax(torch.softmax(out,dim=1),dim=1)
  for key,value in myDataset.labels_map.items():
    if value==pred_label:
      class_name = key
  #print(f"картинка {img_path} \nрезультат предсказания - {class_name}.")
  return pred_label,class_name

In [ ]:
predict_image(model,'/content/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_34.jpg')

In [ ]:
def test_testset(path):
  path = path+'/'
  results = []
  test_classes_names=[]
  model.eval()
  for i in os.listdir(path):
    pred_label,class_name = predict_image(model,os.path.join(path,i))
    p=i[::-1]
    _, p =p.split('_', 1)
    p=p[::-1]
    if class_name not in test_classes_names:
        test_classes_names.append(class_name)
    results.append((p, class_name))
  return results , test_classes_names

In [ ]:
results,_ = test_testset('/content/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset')

In [ ]:
df = pd.DataFrame(results, columns=['Expected', 'Predicted'])

df.to_csv('predictions.csv', index=False)

In [ ]:
df = pd.read_csv('predictions.csv')

In [ ]:
ct = pd.crosstab(df['Expected'],df['Predicted'])

ct

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
true_labels = df['Expected'].values
pred_labels = df['Predicted'].values
cm_array = confusion_matrix(true_labels, pred_labels)

In [ ]:
precision = cm_array.diagonal() / cm_array.sum(axis=0)
recall = cm_array.diagonal() / cm_array.sum(axis=1)

In [ ]:
pres_sum = 0
rec_sum = 0
f1_sum = 0
pres_count = 0
rec_count = 0
f1_count = 0

for i in range(len(recall)):
  if round(precision[i],2)!=0:
    print(f"Class {ct.columns[i]}: Precision={round(precision[i],2)}, Recall={round(recall[i],2)} f1-score = {round(2 * ((recall[i]*precision[i])/(recall[i]+precision[i])),2)}")
    pres_sum += round(precision[i],2)
    rec_sum += round(recall[i],2)
    f1_sum += round(2 * ((recall[i]*precision[i])/(recall[i]+precision[i])),2)
    pres_count += 1
    rec_count += 1
    f1_count += 1


In [ ]:
import numpy as np

print(f"average Precision={round(pres_sum/pres_count, 2)} average Recall={round(rec_sum/rec_count, 2)} average F1-score = { round(f1_sum/f1_count, 2)}")